In [1]:
# Carregando os dados
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression  


# Carregando os dados
df = pd.read_csv('dados/MQD_1465_final.csv', sep=',', encoding='utf-8')
df.head()

,ItemNumber,frase,cla_maj_femi,qtd_maj_femi,cla_maj_masc,qtd_maj_masc,f1_class,f2_class,f3_class,f4_class,...,m3_class,m4_class,f1_tempo,f2_tempo,f3_tempo,f4_tempo,m1_tempo,m2_tempo,m3_tempo,m4_tempo
0,1,Você sabia que o menino que mais vai te dar va...,positiva,4,positiva,4,positiva,positiva,positiva,positiva,...,positiva,positiva,30.117,293.973,21.735,39.664,105.630,16.967,299.204,14.928
1,2,"Hoje com 21 anos, como que por auxilio lá do a...",positiva,4,positiva,3,positiva,positiva,positiva,positiva,...,positiva,positiva,13.839,237.617,83.358,24.249,22.499,9.802,11.472,6.073
2,3,"Nesse momento to evitando ela, e ta me dando u...",negativa,3,negativa,2,negativa,negativa,negativa,neutra,...,negativa,neutra,15.344,13.894,8.584,133.325,20.098,8.170,16.590,13.952
3,4,Meus sentidos de garota diziam que tinha algum...,neutra,2,neutra,3,neutra,negativa,neutra,positiva,...,neutra,neutra,16.747,16.986,10.610,10.337,33.871,11.848,22.167,9.798
4,5,"Aquela promessa que eu te fiz naquele natal, e...",positiva,3,positiva,3,positiva,positiva,negativa,positiva,...,positiva,neutra,8.405,14.191,3.900,5.129,8.789,3.306,5.719,4.554


In [2]:
def check_tie(row, prefix):
    """
    Verifica se há empate nas classificações (2+2)
    Retorna True se houver empate, False caso contrário
    """
    classes = [row[f'{prefix}{i}_class'] for i in range(1, 5)]
    classes = [str(x).lower().strip() for x in classes]
    
    # Contagem de cada classificação
    counts = {
        'positiva': classes.count('positiva'),
        'negativa': classes.count('negativa'),
        'neutra': classes.count('neutra')
    }
    
    # Filtrar apenas contagens diferentes de zero
    counts_values = [v for v in counts.values() if v > 0]
    
    # É empate se tivermos exatamente duas contagens de 2
    return len(counts_values) == 2 and all(count == 2 for count in counts_values)

# Identificar empates para cada gênero
df['empate_f'] = df.apply(lambda row: check_tie(row, 'f'), axis=1)
df['empate_m'] = df.apply(lambda row: check_tie(row, 'm'), axis=1)

# Criar DataFrame apenas com registros que têm empate
df_empates = df[df['empate_f'] | df['empate_m']].copy()

# Mostrar informações sobre os empates
print("Total de registros com empate nas classificações femininas:", df['empate_f'].sum())
print("Total de registros com empate nas classificações masculinas:", df['empate_m'].sum())
print("Total de registros com empate em pelo menos um dos gêneros:", len(df_empates))

# Mostrar as primeiras linhas do DataFrame de empates
print("\nPrimeiras linhas dos registros com empate:")
cols_to_show = ['empate_f', 'empate_m'] + [f'{p}{i}_class' for p in ['f', 'm'] for i in range(1, 5)]
print(df_empates[cols_to_show].head())

Total de registros com empate nas classificações femininas: 227
Total de registros com empate nas classificações masculinas: 163
Total de registros com empate em pelo menos um dos gêneros: 349

Primeiras linhas dos registros com empate:
    empate_f  empate_m   f1_class   f2_class   f3_class   f4_class   m1_class  \
9       True     False   neutra     neutra     negativa   negativa   negativa   
19     False      True   positiva   neutra     positiva   negativa   negativa   
23      True     False   neutra     positiva   positiva   neutra     positiva   
24      True     False   neutra     negativa   negativa   neutra     negativa   
26     False      True   positiva   positiva   positiva   neutra     negativa   

     m2_class   m3_class   m4_class  
9    negativa   negativa   neutra    
19   negativa   positiva   positiva  
23   negativa   positiva   positiva  
24   negativa   negativa   positiva  
26   positiva   positiva   negativa  
